In [37]:
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

In [41]:
df_waw = pd.read_pickle('.\pickle\df_waw_analyse_2.pickle')
df_waw.head()

,dzielnica,od,poziom,umeblowanie,zabudowa,powierzchnia,cena całkowita,powierzchnia na pokoj
0,Targówek,Biuro / Deweloper,2,1,Apartamentowiec,60,2301,30.0
1,Bielany,Osoby prywatnej,3,1,Apartamentowiec,56,3001,28.0
2,Śródmieście,Osoby prywatnej,2,1,Apartamentowiec,29,2201,29.0
3,Mokotów,Osoby prywatnej,2,1,Blok,20,2150,10.0
4,Wawer,Osoby prywatnej,2,1,Apartamentowiec,60,2800,20.0


Najpierw trzeba pozbyć się wsystkich tekstowych wartości z bazy, aby model regresji mógł je uwzględnić.

W kolumnach: "dzielnica", "od", "poziom", "zabudowa", trzeba utworzyć kolumny dummies. Dodatkowo muszę wyrzucić którąś z nowo utworzonych kolumn. W przeciwnym przypadku, model nie będzie w stanie zinterpretować czy zmiana jest spowodowana wartośćią 1 w tej kolumnie czy 0 w pozostałych.   
Przykład: _Apartamentowiec = 1 - Blok - Dom - Kamienica - Szeregowiec_. Kolumna Apartamentowiec jest idealnie zdefiniowana przez pozostałe 4 kolumny. Prowadzi to do wielu problemów w regresji (nieinterpretowalność coeff, współliniowość zmiennych).  
W kolumnach dummies odrzucam: **Mokotów** dla "dzielnicy", **osoby przywatnej** dla "od", **powyżej 10** dla "poziom", **apartamentowiec** dla "zabudowa".

In [62]:
def data_preparation(X_drop):
    
    df_waw = pd.read_pickle('.\pickle\df_waw_analyse_2.pickle')
    dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
    od = pd.get_dummies(data=df_waw['od'])
    poziom = pd.get_dummies(data=df_waw['poziom'])
    zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

    df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)

    y = df_waw['cena całkowita']
    X = df_waw.drop(X_drop, axis=1)
    return [X, y]

In [86]:
X, y= data_preparation(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa','Mokotów',
                        'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'])

In [87]:
def ols_regresion():
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
    
    X_const = sm.add_constant(X_train)
    ols = sm.OLS(y_train, X_const).fit()
    print(ols.summary())

In [88]:
ols_regresion()

                            OLS Regression Results                            
Dep. Variable:         cena całkowita   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     116.0
Date:                Sat, 17 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:37:46   Log-Likelihood:                -21279.
No. Observations:                2818   AIC:                         4.263e+04
Df Residuals:                    2781   BIC:                         4.285e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1795.24

Do usunięcia "powierzchnia na pokoj" (wprowadza multicollinearity), umeblowanie (wysokie p_value)

In [89]:
X, y= data_preparation(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'Mokotów', 'umeblowanie',
                        'powierzchnia na pokoj', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'])
ols_regresion()

                            OLS Regression Results                            
Dep. Variable:         cena całkowita   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     119.0
Date:                Sat, 17 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:37:50   Log-Likelihood:                -21306.
No. Observations:                2818   AIC:                         4.268e+04
Df Residuals:                    2783   BIC:                         4.289e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1558.5381     62.62

In [93]:
X, y= data_preparation(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'Mokotów', 'umeblowanie',
                        'powierzchnia na pokoj', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'])

#standaryzacja danych
scaler = StandardScaler()
X = scaler.fit_transform(X)

ols_regresion()

                            OLS Regression Results                            
Dep. Variable:         cena całkowita   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     119.0
Date:                Sat, 17 Apr 2021   Prob (F-statistic):               0.00
Time:                        19:22:56   Log-Likelihood:                -21306.
No. Observations:                2818   AIC:                         4.268e+04
Df Residuals:                    2783   BIC:                         4.289e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2310.3245      8.837    261.447      0.0

Przeprowadzenie standaryzacji nie wpływa na jakość modelu, natomiast przjerzystość wyników, zwłaszcza współczynnika coef jest gorsza.

Przetworzone X i y zapisuję do pliku .pickle

In [96]:
X, y= data_preparation(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'Mokotów', 'umeblowanie',
                        'powierzchnia na pokoj', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'])

X.to_pickle('./pickle/X.pickle')
y.to_pickle('./pickle/y.pickle')

Przeprowadzam wstępny testy różnych modeli metodą walidacji krzyżowej

In [90]:
linear_regression_cross = cross_val_score(LinearRegression(), X, y)
knn_cross = cross_val_score(KNeighborsRegressor(), X, y)
random_forest_cross = cross_val_score(RandomForestRegressor(), X, y)

print('linear regression', linear_regression_cross)
print('knn regressor', knn_cross)
print('random forest regressor', random_forest_cross)

linear regression [0.57951856 0.57161076 0.63714924 0.48433914 0.57255247]
knn regressor [0.58361357 0.57226998 0.60035742 0.41297776 0.4854528 ]
random forest regressor [0.64219817 0.66759578 0.70914234 0.56912947 0.61189814]


Regresor lasów losowych radzi sobie najlepiej. Model regresji liniowej przewiduje ceny trochę gorzej.

Problemem jest wysoka wariancja poszczególnych wyników walidacji krzyżowej